In [ ]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

# Data Visualiation
import matplotlib.pyplot as plt
import seaborn as sns

# System
import os

# Deep Learning
import tensorflow as tf
from typing import Dict, List, Tuple, Sequence

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.model_selection import train_test_split

In [ ]:
""""list of parameters"""
N_FEATURES = 19
N_TARGETS = 1

FOLD_LENGTH = 8760
FOLD_STRIDE = 728
TRAIN_TEST_RATIO = 0.66

N_TRAIN = 6666
N_TEST = 3333
INPUT_LENGTH = 112
OUTPUT_LENGTH = 1

In [ ]:
import pandas as pd
file_path="~/code/Agermita/intelligent_network_expansion/raw_data/LTE KPIs Part 2.csv"

df_part_2 = pd.read_csv(file_path, sep = ';')
df_part_2.columns

In [ ]:
processed_data.drop('Unnamed: 0', axis=1, inplace=True)
processed_data.drop('Unnamed: 0.1', axis=1, inplace=True)

In [ ]:
def split_train_test(X,y, ratio=0.75) ->tuple:

    X_train, X_test = np.split(X, [int(ratio*len(X))])
    y_train, y_test = np.split(y, [int(ratio*len(y))])
    return X_train, X_test, y_train, y_test

def initialize_model(input_shape: tuple, output_length) -> models:
    #output_length = y_train.shape[1]
    #input_shape =(X.shape[1],X.shape[2])
    """
    Initialize the Neural Network with random weights
    """

    model = models.Sequential()
     ## 1.1 - Recurrent Layer
    model.add(layers.Masking(mask_value=0, input_shape=input_shape))
    model.add(layers.LSTM(units=64,
                          activation='relu',
                          return_sequences = True,
                          kernel_regularizer=L1L2(l1=0.05, l2=0.05),
                          ))
    ## 1.2 - Hidden Dense Layers

    model.add(layers.Dense(32, activation="relu", kernel_regularizer = L1L2(l1=0, l2=0.05)))
    model.add(layers.Dropout(rate=0.5))

    ## 1.2 - Predictive Dense Layers

    model.add(layers.Dense(output_length, activation='linear'))

    return model


def compile_model(model: models, params) -> models:
    """
    Compile the Neural Network
    """

    # 2 - Compiler
    # ======================
    initial_learning_rate = 0.01

    lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=1000, decay_rate=0.5)

    adam = optimizers.Adam(learning_rate=lr_schedule)
    model.compile(loss='mse', optimizer=adam, metrics=["mae"])

    return model

def train_model(
        model: models,
        X: np.ndarray,
        y: np.ndarray,
        patience=2,
        validation_data=None, # overrides validation_split
        validation_split=0.3
    ) -> Tuple[models, dict]:
    """
    Fit the model and return a tuple (fitted_model, history)
    """
     # $CHALLENGIFY_BEGIN
    es = EarlyStopping(monitor = "val_loss",
                      patience = patience,
                      mode = "min",
                      restore_best_weights = True)


    history = model.fit(X, y,
                        validation_split = validation_split,
                        shuffle = False,
                        batch_size = 64,
                        epochs = 50,
                        callbacks = [es],
                        verbose = 1)

    return model, history

In [ ]:
baseline_model = init_baseline()
baseline_score = baseline_model.evaluate(X_test, y_test)
print(f"- The Baseline MAE on the test set is equal to {round(baseline_score[1],2)} Celsius degrees")